# Imports

In [1]:
import sys
sys.path.insert(0, '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/reversed_hgru_with_corr_1 - Norway/model/')

In [2]:
import pandas as pd
import numpy as np
import torch
import pickle
import itertools
import random
from torch.utils.tensorboard import SummaryWriter
import os

from utils import *

2023-06-25 11:49:19.916255: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from pipeline_config import *

# Seeds for Comparisons:

In [4]:
torch.manual_seed(1)
np.random.seed(2)
random.seed(3)

# Read Data

In [5]:
#with open(son_parent_path, 'rb') as f:
#    son_parent_dict = pickle.load(f)
son_parent_dict = pd.read_pickle(son_parent_path)

#with open(train_dataset_dict_path, 'rb') as f:
#    train_dataset_dict = pickle.load(f)
train_dataset_dict = pd.read_pickle(train_dataset_dict_path)

#with open(test_dataset_dict_path, 'rb') as f:
#    test_dataset_dict = pickle.load(f)
test_dataset_dict = pd.read_pickle(test_dataset_dict_path)

#with open(coefficient_dict_path, 'rb') as f:
#    coefficient_dict = pickle.load(f)
coefficient_dict = pd.read_pickle(coefficient_dict_path)

#with open(category_id_to_category_name_path, 'rb') as f:
#    category_id_to_name_dict = pickle.load(f)
category_id_to_name_dict = pd.read_pickle(category_id_to_category_name_path)
    
#with open(categories_per_indent_path, 'rb') as f:
#    categories_per_indent_dict = pickle.load(f)
categories_per_indent_dict = pd.read_pickle(categories_per_indent_path)

#with open(reversed_parent_to_son_list_path, 'rb') as f:
#    parent_to_son_list_dict = pickle.load(f)
parent_to_son_list_dict = pd.read_pickle(reversed_parent_to_son_list_path)

#with open(hgru_weight_path, 'rb') as f:
#    hgru_weight_dict = pickle.load(f)
hgru_weight_dict = pd.read_pickle(hgru_weight_path)

weights_path = weightspath

In [6]:
sorted(list(categories_per_indent_dict.keys()),reverse=True)

[2, 1, 0]

In [7]:
import os
from pathlib import Path
def rmdir(dirc):
    dirc = Path(dirc)
    for itm in dirc.iterdir():
        if itm.is_dir():
            rmdir(itm)
            print("Deleting", itm, ".... successful.")
        else:
            itm.unlink()
    dirc.rmdir()

# Hierarchical GRU

In [8]:
def bi_directional_model(son_parent_dict, parent_to_son_list_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weights_path, hgru_weight_dict, coefficient_dict):
    # deleting files in models_weights folder
    path = os.getcwd()
    desired_path = path + '/models_weights/'
    is_empty = not any(Path(desired_path).iterdir())

    if is_empty==False:
        rmdir(Path(desired_path))
    
    # deleting empty folder
    try:
        os.rmdir(desired_path)
    except OSError:
        print ("Deletion of the directory %s failed" % desired_path)
    else:
        print ("Successfully deleted the directory %s" % desired_path)

    # creating new folder
    try:
        os.mkdir(desired_path)
    except OSError:
        print ("Creation of the directory %s failed" % desired_path)
    else:
        print ("Successfully created the directory %s" % desired_path)

    #-----------------------------------------------------------------------------------------------------------------------------------------------#

    num_categories = 0
    Lr = 0.09368413467461317
    loss_coef_3 = 2.85692265634073*np.exp(-6)

    for indent in sorted(list(categories_per_indent_dict.keys()),reverse=True):
    #for indent in [5]:
        for category in categories_per_indent_dict[indent]:
            num_categories +=1
            print(f'num categories: {num_categories}')
            category_name = category_id_to_name_dict[category]
            print(f'category id|name: {category}|{category_name}')

            if int(indent) == 0 or son_parent_dict[category] not in categories_per_indent_dict[indent-1]: #or (category, son_parent_dict[category]) not in son_parent_corr_dict.keys():
                loss_coef_1 = 0
            else:
                loss_coef_1 = 2.3795175788867531*np.exp(-7) #post optuna

            if (category not in list(parent_to_son_list_dict.keys())) or (set(parent_to_son_list_dict[category]['sons']).issubset(set(categories_per_indent_dict[indent+1]))):
                loss_coef_2 = 0
            else:
                loss_coef_2 = 1.4783276395197786*np.exp(-10) #post optuna
            
            print('------------------------------------------------------------------')

            train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category_name], test_dataset_dict[category_name])
            model = GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb)
            optimizer = torch.optim.AdamW(model.parameters(), lr=Lr)
            model.to(Device)
            saving_param_path = weights_path+category_name+'.pt'
            
            training_and_evaluation(model, indent, train_dataloader, test_dataloader, optimizer, category, hgru_weight_dict, coefficient_dict, son_parent_dict, parent_to_son_list_dict, category_id_to_name_dict, loss_coef_1, loss_coef_2, loss_coef_3, saving_param_path)

In [9]:
bi_directional_models = bi_directional_model(son_parent_dict, parent_to_son_list_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weightspath, hgru_weight_dict, coefficient_dict)

Successfully deleted the directory /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/reversed_hgru_with_corr_1 - Norway/models_weights/
Successfully created the directory /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/reversed_hgru_with_corr_1 - Norway/models_weights/
num categories: 1
category id|name: 45|Accommodation services
------------------------------------------------------------------
num categories: 2
category id|name: 19|Actual rentals for housing
------------------------------------------------------------------
num categories: 3
category id|name: 15|Alcoholic beverages
------------------------------------------------------------------
num categories: 4
category id|name: 38|Audio-visual, photographic and information processing equipment
------------------------------------------------------------------
num categories: 5
category id|name: 17|Clothing
------------------------------------------

In [10]:
def get_results_on_test_set(weights_path, train_dataset_dict, test_dataset_dict, categories = None):
    predictions_dict = {}
    if categories is None:
        categories = list(test_dataset_dict.keys())
        
    for category in categories:
        if category == 'Food and beverages':
            continue
        print(category)
        train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category], test_dataset_dict[category])
        basic_model =GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb)
        basic_optimizer = torch.optim.AdamW(basic_model.parameters(), lr=0.00005)
        ckp_path = weights_path+category+'.pt'
        model, optimizer, checkpoint, valid_loss_min = load_checkpoint(ckp_path, basic_model, basic_optimizer)
        predictions = get_predictions_on_test_set(model, test_dataloader)
        predictions_dict[category] = predictions
    return predictions_dict

In [11]:
categories_lists = list(categories_per_indent_dict.values())
categories_id = list(itertools.chain.from_iterable(categories_lists))
categories = []
for category_id in categories_id:
    categories.append(category_id_to_name_dict[category_id])

In [12]:
categories

['Accommodation services',
 'Actual rentals for housing',
 'Alcoholic beverages',
 'Audio-visual, photographic and information processing equipment',
 'Clothing',
 'Electricity, gas and other fuels',
 'Financial services n.e.c.',
 'Food',
 'Footwear',
 'Furniture and furnishings, carpets and other floor coverings',
 'Glassware, tableware and household utensils',
 'Goods and services for routine household maintenance',
 'Household appliances',
 'Household textiles',
 'Imputed rentals for housing',
 'Insurance',
 'Maintenance and repair of the dwelling',
 'Medical products, appliances and equipment',
 'Newspapers, books and stationery',
 'Non-alcoholic beverages',
 'Operation of personal transport equipment',
 'Other major durables for recreation and culture',
 'Other recreational items and equipment, gardens and pets',
 'Other services n.e.c.',
 'Out-patient services',
 'Package holidays',
 'Personal care',
 'Personal effects n.e.c.',
 'Postal services',
 'Purchase of vehicles',
 'Recre

In [13]:
predictions_dict = get_results_on_test_set(weightspath, train_dataset_dict, test_dataset_dict, categories = categories)

Accommodation services
Actual rentals for housing
Alcoholic beverages
Audio-visual, photographic and information processing equipment
Clothing
Electricity, gas and other fuels
Financial services n.e.c.
Food
Footwear
Furniture and furnishings, carpets and other floor coverings
Glassware, tableware and household utensils
Goods and services for routine household maintenance
Household appliances
Household textiles
Imputed rentals for housing
Insurance
Maintenance and repair of the dwelling
Medical products, appliances and equipment
Newspapers, books and stationery
Non-alcoholic beverages
Operation of personal transport equipment
Other major durables for recreation and culture
Other recreational items and equipment, gardens and pets
Other services n.e.c.
Out-patient services
Package holidays
Personal care
Personal effects n.e.c.
Postal services
Purchase of vehicles
Recreational and cultural services
Restaurant services
Social protection
Telephone equipment
Telephone services
Tobacco
Tools a

In [14]:
len(predictions_dict.keys())

52

In [15]:
with open(test_predictions_path_with_hgru, 'wb') as handle:
    pickle.dump(predictions_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)